In [ ]:
# Does not work
# https://towardsdatascience.com/double-your-google-colab-ram-in-10-seconds-using-these-10-characters-efa636e646ff

# a = [i for i in range(10**10)]
# a[:20]

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd drive/MyDrive/colab_projects/nlp_from_dls/5_HW_Embeddings/

/content/drive/MyDrive/colab_projects/nlp_from_dls/5_HW_Embeddings


In [3]:
%ls

 archive.zip
'Copy of [homework]embeddings.ipynb'
 dev_word2tfidf.pkl
 model_baseline.pt
 model_contex_existing_11.pt
 model_context_11_window3_drop_zero.pt
 model_tfidf.pt
 pca_baseline.png
 pca_context_existing_11.png
 pca_context_naive.png
 test_results_baseline.png
 test_results_context_existing_11.png
 test_results_context_naive.png
 test_word2tfidf.pkl
 training.1600000.processed.noemoticon.csv
 training_baseline.png
 training_context_existing_11.png
 training_context_naive.png


<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

# Embeddings

Привет! В этом домашнем задании мы с помощью эмбеддингов решим задачу семантической классификации твитов.

Для этого мы воспользуемся предобученными эмбеддингами word2vec.

In [4]:
%ls

 archive.zip
'Copy of [homework]embeddings.ipynb'
 dev_word2tfidf.pkl
 model_baseline.pt
 model_contex_existing_11.pt
 model_context_11_window3_drop_zero.pt
 model_tfidf.pt
 pca_baseline.png
 pca_context_existing_11.png
 pca_context_naive.png
 test_results_baseline.png
 test_results_context_existing_11.png
 test_results_context_naive.png
 test_word2tfidf.pkl
 training.1600000.processed.noemoticon.csv
 training_baseline.png
 training_context_existing_11.png
 training_context_naive.png


Для начала скачаем датасет для семантической классификации твитов:

In [5]:
# !gdown https://drive.google.com/uc?id=1eE1FiUkXkcbw0McId4i7qY-L8hH-_Qph&export=download
# !unzip archive.zip

Импортируем нужные библиотеки:

In [6]:
import math
import random
import string

import numpy as np
import pandas as pd
import seaborn as sns

import torch
import nltk
import gensim
import gensim.downloader as api

In [7]:
random.seed(42)
np.random.seed(42)
torch.random.manual_seed(42)
torch.cuda.random.manual_seed(42)
torch.cuda.random.manual_seed_all(42)

device = "cuda" if torch.cuda.is_available() else "cpu"

In [8]:
data = pd.read_csv("training.1600000.processed.noemoticon.csv", encoding="latin",
                   header=None, names=["emotion", "id", "date", "flag", "user", "text"])

Посмотрим на данные:

In [7]:
data.head()

,emotion,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


Выведем несколько примеров твитов, чтобы понимать, с чем мы имеем дело:

In [ ]:
examples = data["text"].sample(10)
print("\n\n".join(examples))

Как вилим, тексты твитов очень "грязные". Нужно предобработать датасет, прежде чем строить для него модель классификации.

Чтобы сравнивать различные методы обработки текста/модели/прочее, разделим датасет на dev(для обучения модели) и test(для получения качества модели).

In [ ]:
indexes = np.arange(data.shape[0])
np.random.shuffle(indexes)
dev_size = math.ceil(data.shape[0] * 0.8)

dev_indexes = indexes[:dev_size]
test_indexes = indexes[dev_size:]

dev_data = data.iloc[dev_indexes]
test_data = data.iloc[test_indexes]

dev_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)

## Обработка текста

Токенизируем текст, избавимся от знаков пунктуации и выкинем все слова, состоящие менее чем из 4 букв:

In [ ]:
tokenizer = nltk.WordPunctTokenizer()
line = tokenizer.tokenize(dev_data["text"][0].lower())
print('Original line:', dev_data["text"][0])
print('Tokens:       ', line)
print('Joined tokens:', " ".join(line))

In [ ]:
filtered_line = [w for w in line if all(c not in string.punctuation for c in w) and len(w) > 3]
print(" ".join(filtered_line))

Загрузим предобученную модель эмбеддингов. 

Если хотите, можно попробовать другую. Полный список можно найти здесь: https://github.com/RaRe-Technologies/gensim-data.

Данная модель выдает эмбеддинги для **слов**. Строить по эмбеддингам слов эмбеддинги предложений мы будем ниже.

In [9]:
%time word2vec = api.load("word2vec-google-news-300")

CPU times: user 1min 47s, sys: 3.22 s, total: 1min 50s
Wall time: 1min 51s


In [11]:
word2vec.vectors.shape

(3000000, 300)

In [ ]:
word2vec['dog'].shape

In [ ]:
word2vec.vector_size

У нас есть эмбединги для 3 миллионов слов. Для каждого слова - 300 фичей.

In [ ]:
emb_line = [word2vec.get_vector(w) for w in filtered_line if w in word2vec]

print('Num words from line in the embedding:', len(emb_line))
print('Shape of the features:', sum(emb_line).shape)

Нормализуем эмбеддинги, прежде чем обучать на них сеть. 
(наверное, вы помните, что нейронные сети гораздо лучше обучаются на нормализованных данных)

In [ ]:
# not all the words are in the 3*10^6 corpus, we need to filter out the missing ones

try:
    print(word2vec['russian'].shape)
    print(80*'-')
    print(word2vec['rassian'].shape)
except KeyError as e:
    print(e)

In [10]:
%%time

# axis=0 - we want mean and standard deviation of each of the 300 features for every word
mean = np.mean(word2vec.vectors, axis=0)
std = np.std(word2vec.vectors, axis=0)

CPU times: user 2.25 s, sys: 2.17 s, total: 4.42 s
Wall time: 4.45 s


In [ ]:
norm_emb_line = [(word2vec.get_vector(w) - mean) / std for w in filtered_line if w in word2vec and len(w) > 3]
print(sum(norm_emb_line).shape)
print([all(norm_emb_line[i] == emb_line[i]) for i in range(len(emb_line))])

In [ ]:
for w in filtered_line:
    if w in word2vec:
        print(((word2vec.get_vector(w) - mean) / std).shape)
        break

Сделаем датасет, который будет по запросу возвращать подготовленные данные.

In [12]:
from torch.utils.data import Dataset, random_split

In [13]:
class TwitterDataset(Dataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec):
        self.tokenizer = nltk.WordPunctTokenizer()
        
        self.data = data

        self.feature_column = feature_column # e.g. text
        self.target_column = target_column   # e.g. emotion

        self.word2vec = word2vec

        self.label2num = lambda label: 0 if label == 0 else 1
        self.mean = np.mean(word2vec.vectors, axis=0)
        self.std = np.std(word2vec.vectors, axis=0)

    def __getitem__(self, item):
        # feature
        text = self.data[self.feature_column][item]
        tokens = self.get_tokens_(text)
        embeddings = self.get_embeddings_(tokens)
        # label
        label = self.label2num(self.data[self.target_column][item])

        return {"feature": embeddings, "target": label}

    def get_tokens_(self, text):
        # Получи все токены из текста и профильтруй их
        tokens = self.tokenizer.tokenize(text.lower())
        filtered_tokens = [t for t in tokens if all(c not in string.punctuation for c in t) and len(t) > 3]
        return filtered_tokens

    def get_embeddings_(self, tokens):
        # Получи эмбеддинги слов и усредни их
        # `and len(t) > 3` is not needed (done during .get_tokens())
        embeddings = [(self.word2vec.get_vector(t) - self.mean) / self.std for t in tokens if t in self.word2vec]
        
        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)

        return embeddings

    def __len__(self):
        return self.data.shape[0]

In [ ]:
dev = TwitterDataset(data=dev_data,
                     feature_column="text", target_column="emotion",
                     word2vec=word2vec)

Отлично, мы готовы с помощью эмбеддингов слов превращать твиты в векторы и обучать нейронную сеть.

Превращать твиты в векторы, используя эмбеддинги слов, можно несколькими способами. А именно такими:

## Average embedding (2 балла)
---
Это самый простой вариант, как получить вектор предложения, используя векторные представления слов в предложении. А именно: вектор предложения есть средний вектор всех слов в предлоежнии (которые остались после токенизации и удаления коротких слов, конечно). 

In [ ]:
indexes = np.arange(len(dev))
np.random.shuffle(indexes)
example_indexes = indexes[::1000]

# changed np.sum to np.mean
examples = {"features": [np.mean(dev[i]["feature"], axis=0) for i in example_indexes], 
            "targets": [dev[i]["target"] for i in example_indexes]}


print(len(examples["features"]), len(examples["features"][0]))

Давайте сделаем визуализацию полученных векторов твитов тренировочного (dev) датасета. Так мы увидим, насколько хорошо твиты с разными target значениями отделяются друг от друга, т.е. насколько хорошо усреднение эмбеддингов слов предложения передает информацию о предложении.

Для визуализации векторов надо получить их проекцию на плоскость. Сделаем это с помощью `PCA`. Если хотите, можете вместо PCA использовать TSNE: так у вас получится более точная проекция на плоскость (а значит, более информативная, т.е. отражающая реальное положение векторов твитов в пространстве). Но TSNE будет работать намного дольше.

In [ ]:
type(examples['features'])

In [11]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=2)
# Обучи PCA на эмбеддингах слов
examples["transformed_features"] = pca.fit_transform(X=examples['features'])

In [12]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

In [ ]:
draw_vectors(
    examples["transformed_features"][:, 0], 
    examples["transformed_features"][:, 1], 
    color=[["red", "blue"][t] for t in examples["targets"]]
    )

Скорее всего, на визуализации нет четкого разделения твитов между классами. Это значит, что по полученным нами векторам твитов не так-то просто определить, к какому классу твит пренадлежит. Значит, обычный линейный классификатор не очень хорошо справится с задачей. Надо будет делать глубокую (хотя бы два слоя) нейронную сеть.

Подготовим загрузчики данных.
Усреднее векторов будем делать в "батчевалке"(`collate_fn`). Она используется для того, чтобы собирать из данных `torch.Tensor` батчи, которые можно отправлять в модель.


In [ ]:
from torch.utils.data import DataLoader


batch_size = 1024
num_workers = 4

def average_emb(batch):
    features = [np.mean(b["feature"], axis=0) for b in batch]
    targets = [b["target"] for b in batch]

    return {"features": torch.FloatTensor(features), "targets": torch.LongTensor(targets)}


In [ ]:
train_size = math.ceil(len(dev) * 0.8)
train, valid = random_split(dev, [train_size, len(dev) - train_size])

train_loader = DataLoader(train, batch_size=batch_size, num_workers=num_workers, shuffle=True, drop_last=True, collate_fn=average_emb)
valid_loader = DataLoader(valid, batch_size=batch_size, num_workers=num_workers, shuffle=False, drop_last=False, collate_fn=average_emb)

In [ ]:
for batch in train_loader:
    print(batch['features'].shape)
    print(batch['targets'].shape)
    print()

    print(batch['features'][:20])
    print(batch['targets'][:20])
    break

Определим функции для тренировки и теста модели:

In [14]:
from tqdm.notebook import tqdm

def training(model, optimizer, criterion, train_loader, epoch, device="cpu"):
    pbar = tqdm(train_loader, desc=f"Epoch {e + 1}. Train Loss: {0}")
    model.train()
    for batch in pbar:
        features = batch["features"].to(device)
        targets = batch["targets"].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(features)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        # print statistics
        pbar.set_description(f"Epoch {e + 1}. Train Loss: {loss:.4}")
    

def testing(model, criterion, test_loader, device="cpu"):
    pbar = tqdm(test_loader, desc=f"Test Loss: {0}, Test Acc: {0}")
    mean_loss = 0
    mean_acc = 0
    model.eval()
    with torch.no_grad():
        for batch in pbar:
            features = batch["features"].to(device)
            targets = batch["targets"].to(device)

            # make predictions and calculate loss
            outputs = model(features)
            loss = criterion(outputs, targets)

            # get class predictions and calculate the accuracy
            _, preds = torch.max(outputs, 1)
            acc = (preds == targets).to(torch.float).mean()            

            mean_loss += loss.item()
            mean_acc += acc.item()

            pbar.set_description(f"Test Loss: {loss:.4}, Test Acc: {acc:.4}")

    pbar.set_description(f"Test Loss: {mean_loss / len(test_loader):.4}, Test Acc: {mean_acc / len(test_loader):.4}")

    return {"Test Loss": mean_loss / len(test_loader), "Test Acc": mean_acc / len(test_loader)}

Создадим модель, оптимизатор и целевую функцию. Вы можете сами выбрать количество слоев в нейронной сети, ваш любимый оптимизатор и целевую функцию.


In [ ]:
import torch.nn as nn
from torch.optim import Adam


# Не забудь поиграться с параметрами ;)
vector_size = dev.word2vec.vector_size
num_classes = 2
lr = 1e-3
num_epochs = 5

# TODO: define the model, loss, and optimiser
model = nn.Sequential(
    nn.Linear(vector_size, 200),
    nn.ReLU(),

    nn.Linear(200, 100),
    nn.ReLU(),

    nn.Linear(100, num_classes),
)
model = model.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

Наконец, обучим модель и протестируем её.

После каждой эпохи будем проверять качество модели на валидационной части датасета. Если метрика стала лучше, будем сохранять модель. **Подумайте, какая метрика (точность или лосс) будет лучше работать в этой задаче?**

У нас здесь стандартная задача классификации на 2 класса. Loss будет работать лучше потому что при уменьшении Loss-a для какого-то конкретного примера, мы будем приближаться к правильному ответу, а значит, будем увеличивать тосность. В отличие от точности, Loss дифференцируем и градиенты будут течь лучше.

In [ ]:
# best_metric = np.inf
# for e in range(num_epochs):
#     training(model, optimizer, criterion, train_loader, e, device)
#     log = testing(model, criterion, valid_loader, device)
#     print(log)
#     if log["Test Loss"] < best_metric:
#         torch.save(model.state_dict(), "model.pt")
#         best_metric = log["Test Loss"]

In [ ]:
test_loader = DataLoader(
    TwitterDataset(test_data, "text", "emotion", word2vec), 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model_baseline.pt", map_location=device))

print(testing(model, criterion, test_loader, device=device))

In [ ]:
%ls

## Embeddings for unknown words (8 баллов)

Пока что использовалась не вся информация из текста. Часть информации фильтровалось – если слова не было в словаре эмбеддингов, то мы просто превращали слово в нулевой вектор. Хочется использовать информацию по-максимуму. Поэтому рассмотрим другие способы обработки слов, которых нет в словаре. А именно:

- Для каждого незнакомого слова будем запоминать его контекст(слова слева и справа от этого слова). Эмбеддингом нашего незнакомого слова будет сумма эмбеддингов всех слов из его контекста. (4 балла)
- Для каждого слова текста получим его эмбеддинг из Tfidf с помощью ```TfidfVectorizer``` из [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer). Итоговым эмбеддингом для каждого слова будет сумма двух эмбеддингов: предобученного и Tfidf-ного. Для слов, которых нет в словаре предобученных эмбеддингов, результирующий эмбеддинг будет просто полученный из Tfidf. (4 балла)

Реализуйте оба варианта **ниже**. Напишите, какой способ сработал лучше и ваши мысли, почему так получилось.

## Embedding of an unknown word = sum of embeddings of the context words

### Эксперимент 1: Naive Context (1 left, 1 right)

**Изменения** в `get_embeddings`

1. Проходимся по всем `tokens`
    1. Если в `word2vec`, добавляем в `embeddings` как в baseline версии нормализуя его по self.mean, self.sdt
    2. Если не в `word2vec`, то заменяем на 'UNK' и добавлеяем нулевой embedding чтобы размер списка embeddings совпадал с размером списка tokens

2. Проходимся еще раз по всем токенам слева направо
    1. Если `token=='UNK'`, заменяем его embedding на среднее между embeddings слева и справа
    2. Только справа для индекса 0, только слева для индекса len(tokens)-1
    3. Нормализуем с self.mean, self.sdt

3. Повторям все как в оригинальной версии после list comprehension

**Code**

```python
class TwitterDatasetContextEmbeddings(TwitterDataset):

    def get_embeddings_(self, tokens):

        embeddings = []
        for i, t in enumerate(tokens):

            # exactly the same as before in the list comprehension
            if t in self.word2vec:
                emb = (self.word2vec.get_vector(t) - self.mean) / self.std
                embeddings.append(emb)
            
            # new part: instead of just skipping a word, represent it through its context
            else:
                tokens[i] = '<UNK>'
                embeddings.append([0 for _ in range(self.word2vec.vector_size)])

        # fill in missing embeddings, not ideal
        for i, t in enumerate(tokens):

            if t == '<UNK>':
                if i == 0:
                    try:
                        emb = embeddings[i+1]
                    except IndexError:
                        # only one token and it's unknown
                        #print("len(tokens):", len(tokens))
                        #print("Tokens:", tokens)
                        emb = [0 for _ in range(self.word2vec.vector_size)]
                elif i == len(tokens)-1:
                    emb = embeddings[i-1]
                else:
                    emb = np.mean([embeddings[i-1], embeddings[i+1]], axis=0)
                
                # normalise and record
                emb = (emb - self.mean) / self.std
                embeddings[i] = emb
            else:
                pass
        
        # from here, everything is the same as before
        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)

        return embeddings
```

**PCA**

Если смотреть на PCA, по PC2 variance уже намного меньше чем по PC1. Все сжалось по 2 компоненту. Скорее всего это произошло потому что до нормализации мы добавили много нулевых/очень маленьких по норме embeddings.

**Проблемa:**

Если у 'UNK' соседи тоже 'UNK', тогда мы заменяем плохо:

Для `[token1, 'UNK', 'UNK', 'UNK', token2]`, мы получим следующие embeddings:


`[emb_t1, emb_t1/2, emb_t1/4, emb_t1/8 + emb_t2/2, emb_t2]`, 
    
а потом еще и нормализуем чтобы быть в отдной размерности со словами из словаря `word2vec`

**Результат:**
    
* Лучшая val   accuracy: 0.619
* Лучшая test  accuracy: 0.588

Намного хуже, чем у baseline модели.

**Вывод**

Надо менять способ получения контекста. Попробуем взять первое ненулевое слово.

----



### Эксперимент 2: Existing Context  (1 left, 1 right)

**Изменения** в `get_embeddings`

1. Проходимся по всем `tokens`
    1. Если в `word2vec`, добавляем в `embeddings` как в baseline версии нормализуя его по self.mean, self.sdt
    2. Если не в `word2vec`, то заменяем на 'UNK' и добавлеяем нулевой embedding чтобы размер списка embeddings совпадал с размером списка tokens

2. Проходимся еще раз по всем токенам слева направо
    1. Если `token=='UNK'`, заменяем его embedding на среднее между embeddings слов слева и справа, которые **не 'UNK'**
    2. Только справа для индекса 0, только слева для индекса len(tokens)-1
    3. Нормализуем с self.mean, self.sdt

3. Повторям все как в оригинальной версии после list comprehension

**Code**

```python
class TwitterDatasetContextEmbeddings(TwitterDataset):


    def get_right_context_word_embedding_(self, tokens, embeddings, i):
        if i == len(tokens):
            return [0 for _ in range(self.word2vec.vector_size)]
        elif tokens[i] != '<UNK>':
            return embeddings[i]
        else:
            return self.get_right_context_word_embedding_(tokens, embeddings, i+1)

    def get_left_context_word_embedding_(self, tokens, embeddings, i):
        if i == -1:
            return [0 for _ in range(self.word2vec.vector_size)]
        elif tokens[i] != '<UNK>':
            return embeddings[i]
        else:
            return self.get_left_context_word_embedding_(tokens, embeddings, i-1)

    def get_embeddings_(self, tokens):

        embeddings = []
        for i, t in enumerate(tokens):

            # exactly the same as before in the list comprehension
            if t in self.word2vec:
                emb = (self.word2vec.get_vector(t) - self.mean) / self.std
                embeddings.append(emb)
            
            # new part: instead of just skipping a word, represent it through its context
            else:
                tokens[i] = '<UNK>'
                embeddings.append(None) # placeholder for indexes

        # fill in missing embeddings, not ideal
        for i, t in enumerate(tokens):

            if t == '<UNK>':

                l_emb = self.get_left_context_word_embedding_(tokens, embeddings, i)
                r_emb = self.get_right_context_word_embedding_(tokens, embeddings, i)

                emb = np.mean([l_emb, r_emb], axis=0)
                
                # normalise and record
                emb = (emb - self.mean) / self.std
                embeddings[i] = emb
            else:
                pass
        
        # from here, everything is the same as before
        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)

        return embeddings
```

**PCA**

PCA стал опять похож на baseline случай.

**Проблемa:**

Если у 'UNK' соседи тоже 'UNK', тогда мы заменяем плохо:

Для `[token1, 'UNK', 'UNK', 'UNK', token2]`, мы получим следующие embeddings:


`[emb_t1, (emb_t1+emb_t1)/2, (emb_t1+emb_t1)/2, (emb_t1+emb_t1)/2, emb_t2]`, 
    

1. Но для самого левого `'UNK'`, `token2` может вообще быть вне контекста.
2. Плюс, `token1` должен быть важнее, чем `token2`.


**Результат:**
    
* Лучшая val   accuracy: 0.743
* Лучшая test  accuracy: 0.747

Чуть хуже, чем у baseline модели. Никакаго overfit-а на validation data не происходит, что хорошо.

**Вывод**

Можно попробовать улучшить способ получения контекста. Попробуем взять первое ненулевое слово в заданном окне.

Q: Что делать если их там нет?
A: Можно просто пропучкать это слово, как делали в baseline версии

In [ ]:
%ls

### Эксперимент 3: Existing Context  (1 left, 1 right inside a pre-specified window=3 of context)

**Изменения** в `get_embeddings`

1. Проходимся по всем `tokens`
    1. Если в `word2vec`, добавляем в `embeddings` как в baseline версии нормализуя его по self.mean, self.sdt
    2. Если не в `word2vec`, то заменяем на 'UNK' и добавлеяем нулевой embedding чтобы размер списка embeddings совпадал с размером списка tokens

2. Проходимся еще раз по всем токенам слева направо
    1. Если `token=='UNK'`, заменяем его embedding на среднее между embeddings слов слева и справа внутри окна ширины 3, которые **не 'UNK'**
        * Только справа для индекса 0, только слева для индекса len(tokens)-1
        * Если получили ненулевой эмбеддинг
            * Нормализуем с self.mean, self.sdt
            * Добавляем индекс в **список с полезными индексами**
    2. Если `token!='UNK'`, то добавляем индекс в **список с полезными индексами**

3. Повторям логику из оригинальной версии после list comprehension

**Code**

```python
class TwitterDatasetContextEmbeddings(TwitterDataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec,
                 emb_window):
        super().__init__(data, feature_column, target_column, word2vec)
        self.emb_window = emb_window

    def get_right_context_word_embedding_(self, tokens, embeddings, i, r_bound):
        if i == min(len(tokens), r_bound+1):
            return np.zeros(self.word2vec.vector_size)
        elif tokens[i] != '<UNK>':
            return embeddings[i]
        else:
            return self.get_right_context_word_embedding_(tokens, embeddings, i+1, r_bound)

    def get_left_context_word_embedding_(self, tokens, embeddings, i, l_bound):
        if i == max(-1, l_bound):
            return np.zeros(self.word2vec.vector_size)
        elif tokens[i] != '<UNK>':
            return embeddings[i]
        else:
            return self.get_left_context_word_embedding_(tokens, embeddings, i-1, l_bound)

    def get_embeddings_(self, tokens):

        indexes_to_keep = []
        embeddings = []
        for i, t in enumerate(tokens):

            # exactly the same as before in the list comprehension
            if t in self.word2vec:
                emb = (self.word2vec.get_vector(t) - self.mean) / self.std
                embeddings.append(emb)
            
            # new part: instead of just skipping a word, represent it through its context
            else:
                tokens[i] = '<UNK>'
                embeddings.append(np.zeros(self.word2vec.vector_size)) # placeholder for indexes


        # fill in missing embeddings, not ideal, but even better
        for i, t in enumerate(tokens):

            if t == '<UNK>':

                l_emb = self.get_left_context_word_embedding_(tokens, embeddings, i, i-self.emb_window)
                r_emb = self.get_right_context_word_embedding_(tokens, embeddings, i, i+self.emb_window)

                emb = np.mean([l_emb, r_emb], axis=0)

                # record indexes of tokes to completely remove
                if all(emb < 1e-8):
                    pass
                else: 
                    # normalise, record, and note that we'll keep
                    emb = (emb - self.mean) / self.std
                    embeddings[i] = emb
                    indexes_to_keep.append(i)

            else:
                indexes_to_keep.append(i)
        
        # keep only relevant indormation if there is any
        if len(indexes_to_keep) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            indexes_to_keep = np.array(indexes_to_keep)
            embeddings = np.array(embeddings)
            embeddings = embeddings[indexes_to_keep]
        
            # reshape if necessary
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)
        

        return embeddings
```

**PCA**

PCA опять похож на baseline случай.

**Проблемa:**

Если у 'UNK' соседи тоже 'UNK', тогда мы заменяем плохо:

Для `[token1, 'UNK', 'UNK', 'UNK', token2]`, мы получим следующие embeddings:


`[emb_t1, (emb_t1+emb_t1)/2, (emb_t1+emb_t1)/2, (emb_t1+emb_t1)/2, emb_t2]`, 
    
Из прошлого эксперимента: для левого `'UNK'`, `token1` должен быть важнее, чем `token2`. Нужно добавить веса, а еще можно учитывать всех знакомых (не `'UNK'`) соседей внутри окна.


**Результат:**
    
* Лучшая val   accuracy: 0.744
* Лучшая test  accuracy: 0.750

Опять чуть хуже, чем у baseline модели. Никакаго overfit-а на validation data не происходит, что хорошо.

Результаты лучше, чем в эксперименте 2, но, на мой взгляд, в пределах погрешности.

**Вывод**

Можно попробовать еще улучшить способ получения контекста. Попробуем взять все ненулевые слова в заданном окне а еще и взвесить их. Сумма весов должна быть 1.

### Эксперимент 4: Existing Context  (all left, all right inside a pre-specified window=3 of context weighted inversly proportional to distance from the central word)

**Изменения** в `get_embeddings`

1. Проходимся по всем `tokens`
    1. Если в `word2vec`, добавляем в `embeddings` как в baseline версии нормализуя его по self.mean, self.sdt
    2. Если не в `word2vec`, то заменяем на 'UNK' и добавлеяем нулевой embedding чтобы размер списка embeddings совпадал с размером списка tokens

2. Проходимся еще раз по всем токенам слева направо
    1. Если `token=='UNK'`, заменяем его embedding на среднее между dctvb embeddings слов слева и справа внутри окна ширины 3, которые **не 'UNK'** взвешивая обратно-пропорционально расстоянию от центрального слова.
        * Только справа для индекса 0, только слева для индекса len(tokens)-1
        * Если получили ненулевой эмбеддинг
            * Нормализуем с self.mean, self.sdt
            * Добавляем индекс в **список с полезными индексами**
    2. Если `token!='UNK'`, то добавляем индекс в **список с полезными индексами**

3. Повторям логику из оригинальной версии после list comprehension

**Code**

```python

class TwitterDatasetContextEmbeddings(TwitterDataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec,
                 emb_window):
        super().__init__(data, feature_column, target_column, word2vec)
        self.emb_window = emb_window
    

    def get_embeddings_(self, tokens):

        indexes_to_keep = []
        embeddings = np.zeros((len(tokens), self.word2vec.vector_size))

        for i, t in enumerate(tokens):

            # exactly the same as before in the list comprehension
            if t in self.word2vec:
                emb = (self.word2vec.get_vector(t) - self.mean) / self.std
                embeddings[i] = emb
            
            # new part: instead of just skipping a word, represent it through its context
            else:
                tokens[i] = '<UNK>'

        # fill in missing embeddings, close to ideal, but messy implementation
        for i, t in enumerate(tokens):

            if t == '<UNK>':

                l_bound = max(0, i-self.emb_window)
                r_bound = min(len(tokens)-1, i+self.emb_window)
                context_inds = np.array([ind for ind in range(l_bound, r_bound+1) if ind != i])

                if len(context_inds) != 0:
                    context = embeddings[context_inds].copy()

                    # do not include unknown words as context
                    for ci in range(len(context)):
                        if tokens[context_inds[ci]] == '<UNK>':
                            context[ci] = np.zeros(self.word2vec.vector_size)

                    # left weights
                    left_weights = []
                    w = 1
                    for _ in range(l_bound, i):
                        left_weights.append(w)
                        w +=1
                    # right weights
                    right_weights = []
                    w = 1
                    for _ in range(i+1, r_bound+1):
                        right_weights.append(w)
                        w +=1
                    right_weights.reverse()
                    # all weights
                    weights = np.array(left_weights + right_weights)
                    weights = weights / weights.sum()
                    weights = weights.reshape(-1, 1)
                    
                    # check
                    assert weights.shape[0] == context.shape[0]

                    emb = np.mean(weights * context, axis=0)
                else:
                    emb = np.zeros(self.word2vec.vector_size)

                # record indexes of tokes with meaningful embeddings
                if all(emb < 1e-8):
                    pass
                else: 
                    # normalise, record, and note that we'll keep
                    emb = (emb - self.mean) / self.std
                    embeddings[i] = emb
                    indexes_to_keep.append(i)

            else:
                indexes_to_keep.append(i)
        
        # keep only relevant indormation if there is any
        if len(indexes_to_keep) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            indexes_to_keep = np.array(indexes_to_keep)
            embeddings = np.array(embeddings)
            embeddings = embeddings[indexes_to_keep]
        
            # reshape if necessary
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)
        
        return embeddings

```

**PCA**

PCA опять похож на baseline случай.


**Результат:**
    
* Лучшая val   accuracy: 0.755
* Лучшая test  accuracy: 0.729

Опять чуть хуже, чем у baseline модели. Пошел overfit-а на validation data. Пора заканчивать с этим. Baseline так и не получилось побить.

**Вывод**

Пора пробовать tf-idf метод :)

### Final Code

In [15]:
class TwitterDatasetContextEmbeddings(TwitterDataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec,
                 emb_window):
        super().__init__(data, feature_column, target_column, word2vec)
        self.emb_window = emb_window
    

    def get_embeddings_(self, tokens):

        indexes_to_keep = []
        embeddings = np.zeros((len(tokens), self.word2vec.vector_size))

        for i, t in enumerate(tokens):

            # exactly the same as before in the list comprehension
            if t in self.word2vec:
                emb = (self.word2vec.get_vector(t) - self.mean) / self.std
                embeddings[i] = emb
            
            # new part: instead of just skipping a word, represent it through its context
            else:
                tokens[i] = '<UNK>'

        # fill in missing embeddings, close to ideal, but messy implementation
        for i, t in enumerate(tokens):

            if t == '<UNK>':

                l_bound = max(0, i-self.emb_window)
                r_bound = min(len(tokens)-1, i+self.emb_window)
                context_inds = np.array([ind for ind in range(l_bound, r_bound+1) if ind != i])

                if len(context_inds) != 0:
                    context = embeddings[context_inds].copy()

                    # do not include unknown words as context
                    for ci in range(len(context)):
                        if tokens[context_inds[ci]] == '<UNK>':
                            context[ci] = np.zeros(self.word2vec.vector_size)

                    # left weights
                    left_weights = []
                    w = 1
                    for _ in range(l_bound, i):
                        left_weights.append(w)
                        w +=1
                    # right weights
                    right_weights = []
                    w = 1
                    for _ in range(i+1, r_bound+1):
                        right_weights.append(w)
                        w +=1
                    right_weights.reverse()
                    # all weights
                    weights = np.array(left_weights + right_weights)
                    weights = weights / weights.sum()
                    weights = weights.reshape(-1, 1)
                    
                    # check
                    assert weights.shape[0] == context.shape[0]

                    emb = np.mean(weights * context, axis=0)
                else:
                    emb = np.zeros(self.word2vec.vector_size)

                # record indexes of tokes with meaningful embeddings
                if all(emb < 1e-8):
                    pass
                else: 
                    # normalise, record, and note that we'll keep
                    emb = (emb - self.mean) / self.std
                    embeddings[i] = emb
                    indexes_to_keep.append(i)

            else:
                indexes_to_keep.append(i)
        
        # keep only relevant indormation if there is any
        if len(indexes_to_keep) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            indexes_to_keep = np.array(indexes_to_keep)
            embeddings = np.array(embeddings)
            embeddings = embeddings[indexes_to_keep]
        
            # reshape if necessary
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)
        
        return embeddings

In [ ]:
dev = TwitterDatasetContextEmbeddings(
    data=dev_data,
    feature_column="text",
    target_column="emotion",
    word2vec=word2vec,
    emb_window=3)

In [ ]:
indexes = np.arange(len(dev))
np.random.shuffle(indexes)
example_indexes = indexes[::1000]

# changed np.sum to np.mean
examples = {"features": [np.mean(dev[i]["feature"], axis=0) for i in example_indexes], 
            "targets": [dev[i]["target"] for i in example_indexes]}


print(len(examples["features"]), len(examples["features"][0]))

In [ ]:
pca = PCA(n_components=2)
# Обучи PCA на эмбеддингах слов
examples["transformed_features"] = pca.fit_transform(X=examples['features'])

In [ ]:
draw_vectors(
    examples["transformed_features"][:, 0], 
    examples["transformed_features"][:, 1], 
    color=[["red", "blue"][t] for t in examples["targets"]]
    )

In [ ]:
from torch.utils.data import DataLoader


batch_size = 1024
num_workers = 4

def average_emb(batch):
    features = [np.mean(b["feature"], axis=0) for b in batch]
    targets = [b["target"] for b in batch]

    return {"features": torch.FloatTensor(features), "targets": torch.LongTensor(targets)}


train_size = math.ceil(len(dev) * 0.8)

train, valid = random_split(dev, [train_size, len(dev) - train_size])

train_loader = DataLoader(train, batch_size=batch_size, num_workers=num_workers, shuffle=True, drop_last=True, collate_fn=average_emb)
valid_loader = DataLoader(valid, batch_size=batch_size, num_workers=num_workers, shuffle=False, drop_last=False, collate_fn=average_emb)

In [ ]:
import torch.nn as nn
from torch.optim import Adam


# TODO: try different parameters
vector_size = dev.word2vec.vector_size
num_classes = 2
lr = 1e-3
num_epochs = 5

# TODO: define the model, loss, and optimiser
model = nn.Sequential(
    nn.Linear(vector_size, 200),
    nn.ReLU(),

    nn.Linear(200, 100),
    nn.ReLU(),

    nn.Linear(100, num_classes),
)
model = model.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [ ]:
best_metric = np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, device)
    log = testing(model, criterion, valid_loader, device)
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model_context_11_window3_drop_zero.pt")
        best_metric = log["Test Loss"]

In [ ]:
test_loader = DataLoader(
    TwitterDatasetContextEmbeddings(test_data, "text", "emotion", word2vec, emb_window=3), 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model_context_11_window3_drop_zero.pt", map_location=device))

print(testing(model, criterion, test_loader, device=device))

## TF-IDF embeddings

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.utils.extmath import randomized_svd

In [16]:
def dummy_fun(doc):
    return doc

tfidf = TfidfVectorizer(
    analyzer='word',
    tokenizer=dummy_fun,
    preprocessor=dummy_fun,
    token_pattern=None)

In [17]:
def my_get_tokens_(text, my_tokenizer=nltk.WordPunctTokenizer()):
    # Получи все токены из текста и профильтруй их
    tokens = my_tokenizer.tokenize(text.lower())
    filtered_tokens = [t for t in tokens if all(c not in string.punctuation for c in t) and len(t) > 3]
    return filtered_tokens

### Get TF-IDF embedding for 1000 examples

In [ ]:
examples = data["text"].sample(1000)
print("\n\n".join(examples[::100]))

In [ ]:
examples

In [ ]:
docs = [my_get_tokens_(text) for text in examples]
docs[::100]

In [ ]:
tfidf_matrix = tfidf.fit_transform(docs)
print(tfidf.vocabulary_)

tfidf_matrix.todense().shape

In [ ]:
tfidf_matrix.todense()[0].shape

### Get a 300-component SVD.

Use:

$$X = V \Sigma U^T$$

instead of the usual form

$$X = U \Sigma V^T$$

**Why?**

U is in our convention a matrix with word-vectors, while V - matrix with document/context vectors.


In [ ]:
V, Sigma, UT = randomized_svd(tfidf_matrix, 
                              n_components=300,
                              n_iter=5,
                              random_state=None)

In [ ]:
V.shape

In [ ]:
Sigma.shape

In [ ]:
UT.shape

In [ ]:
UT[:, 0].shape

### Get TF-IDF features for vocabulary in the example documents

In [ ]:
exampleword2tfidf = {}
for v in tfidf.vocabulary_:
    exampleword2tfidf[v] = UT[:, tfidf.vocabulary_[v]]

In [ ]:
exampleword2tfidf['make'].shape

### Get TF-IDF embeddings for dev and test sets

In [18]:
indexes = np.arange(data.shape[0])
np.random.shuffle(indexes)
dev_size = math.ceil(data.shape[0] * 0.8)

dev_indexes = indexes[:dev_size]
test_indexes = indexes[dev_size:]

dev_data = data.iloc[dev_indexes]
test_data = data.iloc[test_indexes]

dev_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)

#### dev set

Uncomment when doing first time. Takes time and RAM, so was precomputed separately and saved.

In [19]:
# dev_examples = dev_data['text']
# dev_docs = [my_get_tokens_(text) for text in dev_examples]

In [20]:
# # X = V Sigma U^T
# UT = randomized_svd(tfidf.fit_transform(dev_docs),
#                     n_components=200,
#                     )[2]

In [ ]:
# dev_word2tfidf = {}
# for v in tfidf.vocabulary_:
#     dev_word2tfidf[v] = UT[:, tfidf.vocabulary_[v]]

In [ ]:
# import pickle

# f = open("dev_word2tfidf.pkl","wb")
# pickle.dump(dev_word2tfidf,f)
# f.close()

In [ ]:
# len(dev_word2tfidf['make'])

#### test set

Uncomment when doing first time. Takes time and RAM, so was precomputed separately and saved.



In [ ]:
# test_examples = test_data['text']
# test_docs = [my_get_tokens_(text) for text in test_examples]

In [ ]:
# # X = V Sigma U^T, so we only need the UT component with index 2
# #
# # we do not fit a new one, but use the fitted on the dev set
# UT = randomized_svd(tfidf.transform(test_docs),
#                     n_components=200,
#                     )[2]

In [ ]:
# test_word2tfidf = {}
# for v in tfidf.vocabulary_:
#     test_word2tfidf[v] = UT[:, tfidf.vocabulary_[v]]

In [ ]:
# import pickle

# f = open("test_word2tfidf.pkl","wb")
# pickle.dump(test_word2tfidf,f)
# f.close()

In [ ]:
# len(test_word2tfidf['make'])

In [ ]:
# # check: they should have different representation because the context and documents in test
# # are different
# any(test_word2tfidf['make'] == dev_word2tfidf['make'])

### TwitterDataset class for reference

```python

class TwitterDataset(Dataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec):
        self.tokenizer = nltk.WordPunctTokenizer()
        
        self.data = data

        self.feature_column = feature_column # e.g. text
        self.target_column = target_column   # e.g. emotion

        self.word2vec = word2vec

        self.label2num = lambda label: 0 if label == 0 else 1
        self.mean = np.mean(word2vec.vectors, axis=0)
        self.std = np.std(word2vec.vectors, axis=0)

    def __getitem__(self, item):
        # feature
        text = self.data[self.feature_column][item]
        tokens = self.get_tokens_(text)
        embeddings = self.get_embeddings_(tokens)
        # label
        label = self.label2num(self.data[self.target_column][item])

        return {"feature": embeddings, "target": label}

    def get_tokens_(self, text):
        # Получи все токены из текста и профильтруй их
        tokens = self.tokenizer.tokenize(text.lower())
        filtered_tokens = [t for t in tokens if all(c not in string.punctuation for c in t) and len(t) > 3]
        return filtered_tokens

    def get_embeddings_(self, tokens):
        # Получи эмбеддинги слов и усредни их
        # `and len(t) > 3` is not needed (done during .get_tokens())
        embeddings = [(self.word2vec.get_vector(t) - self.mean) / self.std for t in tokens if t in self.word2vec]
        
        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)

        return embeddings

    def __len__(self):
        return self.data.shape[0]

```

### TwitterDatasetTFIDFEmbeddings class

In [21]:
class TwitterDatasetTFIDFEmbeddings(TwitterDataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec,
                 word2tfidf):
        super().__init__(data, feature_column, target_column, word2vec)
        self.word2tfidf = word2tfidf


    def get_embeddings_(self, tokens):
        # Получи эмбеддинги слов и усредни их

        embeddings_pretrained = np.zeros((len(tokens), self.word2vec.vector_size))
        embeddings_tfidf = np.zeros((len(tokens), 200)) # 300 did not fit into RAM
        
        for i, t in enumerate(tokens):
            
            if t in self.word2tfidf:
                embeddings_tfidf[i] = self.word2tfidf.get(t)
            if t in self.word2vec:
                embeddings_pretrained[i] = (self.word2vec.get_vector(t) - self.mean) / self.std

        embeddings = np.hstack([embeddings_pretrained, embeddings_tfidf])

        assert embeddings.shape[1] == 500

        if len(embeddings) == 0:
            embeddings = np.zeros((1, 500))
        else:
            embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)

        return embeddings

### Final training

In [104]:
# import pickle

# with open('dev_word2tfidf.pkl', 'rb') as f:
#     word2tfidf = pickle.load(f)

In [22]:
dev = TwitterDatasetTFIDFEmbeddings(
    data=dev_data,
    feature_column="text", target_column="emotion",
    word2vec=word2vec,
    word2tfidf=word2tfidf)

NameError: ignored

In [23]:
len(dev)

NameError: ignored

In [ ]:
indexes = np.arange(len(dev))
np.random.shuffle(indexes)
example_indexes = indexes[::1000]

# changed np.sum to np.mean
examples = {"features": [np.mean(dev[i]["feature"], axis=0) for i in example_indexes], 
            "targets": [dev[i]["target"] for i in example_indexes]}


print(len(examples["features"]), len(examples["features"][0]))

In [24]:
from torch.utils.data import DataLoader

batch_size = 1024
num_workers = 4

def average_emb(batch):
    features = [np.mean(b["feature"], axis=0) for b in batch]
    targets = [b["target"] for b in batch]

    return {"features": torch.FloatTensor(features), "targets": torch.LongTensor(targets)}


train_size = math.ceil(len(dev) * 0.8)

train, valid = random_split(dev, [train_size, len(dev) - train_size])

train_loader = DataLoader(train, batch_size=batch_size, num_workers=num_workers, shuffle=True, drop_last=True, collate_fn=average_emb)
valid_loader = DataLoader(valid, batch_size=batch_size, num_workers=num_workers, shuffle=False, drop_last=False, collate_fn=average_emb)

NameError: ignored

In [25]:
import torch.nn as nn
from torch.optim import Adam


# TODO: try different parameters

# 300 from word2vec
# 200 from tfidf
vector_size = 500

num_classes = 2
lr = 1e-3
num_epochs = 5

# TODO: define the model, loss, and optimiser
model = nn.Sequential(
    nn.Linear(vector_size, 200),
    nn.ReLU(),

    nn.Linear(200, 100),
    nn.ReLU(),

    nn.Linear(100, num_classes),
)
model = model.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [111]:
best_metric = np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, device)
    log = testing(model, criterion, valid_loader, device)
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model_tfidf.pt")
        best_metric = log["Test Loss"]


{'Test Loss': 0.49779582488536833, 'Test Acc': 0.75541015625}



{'Test Loss': 0.49519594275951384, 'Test Acc': 0.757203125}



{'Test Loss': 0.4908740626573563, 'Test Acc': 0.760125}



{'Test Loss': 0.4914547641277313, 'Test Acc': 0.75971875}



{'Test Loss': 0.4869244567155838, 'Test Acc': 0.762765625}


In [27]:
import pickle

# rewrite word2tfidf to save RAM
with open('test_word2tfidf.pkl', 'rb') as f:
    word2tfidf = pickle.load(f)

# create test loader
# note, we create the dataset inside
test_loader = DataLoader(
    TwitterDatasetTFIDFEmbeddings(test_data, "text", "emotion", word2vec,
                                  word2tfidf=word2tfidf), 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model_tfidf.pt", map_location=device))

print(testing(model, criterion, test_loader, device=device))


{'Test Loss': 0.5195554243489957, 'Test Acc': 0.7385745307507987}


### Итоги с TFIDF

Лучшие:

* Valid Accuracy: 0.763
* Test Accuracy : 0.739

Validation accuracy лучше, чем когда либо, а вот Test accuracy упала. Произошёл overfit на dev set.

## Общий Вывод:

Тяжело тренить, когда RAM не хватает :(

Поэтому в ответах на печать будут только последние результаты с TFIDF.

Делать фичи слова из контекста (первое улучшение бещ TF-IDF) вообще мне кажется не очень хорошей идеей потому что мы пытаемся взять линейную комбинацию фичей, которые у нас и так есть. ничего хорошего из этого не вышло.

TFIDF поинтереснее, но тоже не дало улучшения. Можно бы еще с ним поковыряться, но пора сдавать, смотреть лекции по RNN, и начинать по ним домашку... Много всего :) 
